In [16]:
import pandas as pd
import preprocessor as p
import re
import numpy as np

import string
import nltk

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

import stop_words
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from textblob import Word
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

stop_words = stop_words.get_stop_words('en') + nltk.corpus.stopwords.words('english')
stop_words = list(set(list(ENGLISH_STOP_WORDS) + stop_words + list(string.ascii_lowercase)))

In [17]:

def remove_links(tweet):
    tweet = re.sub(r'http\S+', ' ', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', ' ', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', ' ', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', ' ', tweet) # remove tweeted at
    return tweet

def clean_tweets(tweet):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.encode('ascii', 'ignore').decode('ascii') # remove emojis
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+string.punctuation+ ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')if word not in stop_words and len(word)>2] # remove stopwords
#     tweet_token_list = [word_rooter(word) if '#' not in word else word for word in tweet_token_list] # apply word rooter
#     if bigrams:
#         tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
#                                             for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    tweet = p.clean(tweet)
#     tweet = word_tokenize(tweet)
    return tweet

df = pd.read_csv("tweets_info.csv")
df = df.fillna('')
df["processed_tweet_text"] = df.tweet_text.map(clean_tweets)
lemmatizer = WordNetLemmatizer()
wordNet_lemmatizer = lambda x: " ".join([ lemmatizer.lemmatize(word) for word in x.split()])
texblob_lemmatizer = lambda x: " ".join([Word(word).lemmatize() for word in x.split()])
df['processed_tweet_text'] = df['processed_tweet_text'].apply(wordNet_lemmatizer)
df['processed_tweet_text'] = df['processed_tweet_text'].apply(texblob_lemmatizer)
# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(df['processed_tweet_text']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()


In [18]:
df

,Unnamed: 0,followers_screen_name,tweet_text,harsh_tags,user_description,followers_description,friends_description,mentions_descriptions,retweeted_descriptions,processed_tweet_text
0,0,AmyVellucci,"Raw video: Amaya Linsey’s brother, First Class...",UTgrad2018,"City Editor, Knoxville News Sentinel. Married,...",News Sentinel tweets by reporter Amy McRary. A...,True Blue news & views from Middle Tennessee S...,The official Twitter of the University of Tenn...,Believer. Husband. Father. Politics reporter f...,raw video amaya linseys brother class jerald l...
1,1,AntwiBenjamin4,HEARTS 4 🆚️ 0 KOTOKO \n\n📍Ghana PL 🇬🇭\n📆15 Mar...,NSMQRegionals2019 NSMQ2019 RMCity HalaMadrid H...,,Men at war never fears bullets life is so swee...,Founder & President of @TPUSA Proud capitalist...,Manages @GHPLLive||@Ghpl_Goals||@ArcGISPro Ana...,Recounting Africa's football history Founder &...,heart kotoko ghana mar accra stadium goal fran...
2,2,JNORMAND1,Over #3rdHourTODAY @_chandman I bet you really...,3rdHourTODAY TitanUp WingsUp stillitcity boomt...,,"Best #writers online! Husband, Father of 4 bea...",The official Twitter of the University of Tenn...,Straight outta the Bible Belt | BBN | Geronimo...,Information on athletics at Oak Ridge High Sch...,rdhourtoday chandman bet really hate losing co...
3,3,RachelTrotter9,@StormTrackBrian https://t.co/1ybD4luN5e @abc1...,SchoolPatrol BLACKPINKonGMA BLACKPINK 블랙핑크 INY...,,1/2: @zindeeofficial “Just As I Am” out now 🎶 ...,The Official East Tennessee State University T...,Meteorologist for Newschannel 9 in Chattanooga...,As seen on TV. Tweeting freely about news & e...,jeffreee star liked think test got failing gra...
4,4,sandypa60908493,,,,dairy beef and grain farmer in warren co 3 son...,The Official East Tennessee State University T...,,,
